In [31]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [21]:
dvfs_list = ['1', '0c00', '0d00', '0e00', 
             '0f00', '1000', '1100', '1200', 
             '1300', '1400', '1500', '1600', '1700', '1800', '1900', '1a00']
dvfs_dict = {
    "0x0c00" :  1.2,
    "0x0d00" :  1.3,
    "0x0e00" :  1.4,
    "0x0f00" :  1.5,
    "0x1000" : 1.6,
    "0x1100" : 1.7,
    "0x1200" : 1.8,
    "0x1300" : 1.9,
    "0x1400" : 2.0,
    "0x1500" : 2.1,
    "0x1600" : 2.2,
    "0x1700" : 2.3,
    "0x1800" : 2.4,
    "0x1900" : 2.5,
    "0x1a00" : 2.6,
    "0x1b00" : 2.7,
    "0x1c00" : 2.8,
    "0x1d00" : 2.9,
}

df_dict = {
    'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 
    
    'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [],'c3Cnt': [], 'c6Cnt': [], 
    'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
    
    'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 
    'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 
    'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 
    'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 
    'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 

    'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 
    'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 
    'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 
    'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 
    'SourcebusyTime_%': [], 'SourcebackPressuredTime_%': [], 

    'MappernumRecordsInPerSecond_avg': [], 
    'MappernumRecordsInPerSecond_std': [], 'MappernumRecordsOutPerSecond_avg': [], 
    'MappernumRecordsOutPerSecond_std': [], 'MapperbusyTimeMsPerSecond_avg': [], 
    'MapperbusyTimeMsPerSecond_std': [], 'MapperbackPressuredTimeMsPerSecond_avg': [], 
    'MapperbackPressuredTimeMsPerSecond_std': [],
    'MapperbusyTime_%': [], 'MapperbackPressuredTime_%': []
}

print(df_dict)
print("*****************************************************************")

def parseFile(rate, itr, dvfs, policy, i):
    file=f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/summary.csv"
    print(file)
    df_dict['i'].append(i)
    df_dict['itr'].append(itr)
    if '0x'+dvfs in dvfs_dict:
        df_dict['dvfs'].append(dvfs_dict['0x'+dvfs])
    else:
        df_dict['dvfs'].append(dvfs)
    df_dict['policy'].append(policy)
    df_dict['rate'].append(rate)
        
    df = pd.read_csv(file)
    df = df[df.columns.drop(list(df.filter(regex='Cnt')))]
    df = df[df.columns.drop(list(df.filter(regex='Bytes')))]
    #print(df)
    
    dff = df[df['name'].str.contains('Sink')]
    dff.columns = 'Sink' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Source')]
    dff.columns = 'Source' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Mapper')]
    dff.columns = 'Mapper' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    jfile = f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/server2_rapl.log"
    with open(jfile) as file:
        lines = [float(line.rstrip()) for line in file]
        print(len(lines))
        df_dict['watts_avg'].append(float(round(np.mean(lines[300:500]), 2)))
        df_dict['watts_std'].append(float(round(np.std(lines[300:500]), 2)))
        #print(np.std(lines[300:500]))
        
    jfile = f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/stats.csv"
    with open(jfile) as file:
        #$POLL, $C1, $C1E, $C3, $C6, $RXP, $RXB, $TXP, $TXB
        # 'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [],'c3Cnt': [], 'c6Cnt': [], 'rxBytes': [], 'txBytes': [],
        poll = []
        c1 = []
        c1e = []
        c3 = []
        c6 = []
        rxp = []
        rxb = []
        txp = []
        txb = []
        for line in file:
            ll = [int(a) for a in line.strip().split(',')]
            poll.append(ll[0])
            c1.append(ll[1])
            c1e.append(ll[2])
            c3.append(ll[3])
            c6.append(ll[4])
            rxp.append(ll[5])
            rxb.append(ll[6])
            txp.append(ll[7])
            txb.append(ll[8])
        df_dict['pollCnt'].append(np.sum(poll[30:50]))
        df_dict['c1Cnt'].append(np.sum(c1[30:50]))
        df_dict['c1eCnt'].append(np.sum(c1e[30:50]))
        df_dict['c3Cnt'].append(np.sum(c3[30:50]))
        df_dict['c6Cnt'].append(np.sum(c6[30:50]))
        #'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
        df_dict['rxPackets'].append(np.sum(rxp[30:50]))
        df_dict['rxBytes'].append(np.sum(rxb[30:50]))
        df_dict['txPackets'].append(np.sum(txp[30:50]))
        df_dict['txBytes'].append(np.sum(txb[30:50]))
  

{'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [], 'c3Cnt': [], 'c6Cnt': [], 'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [], 'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 'SourcebusyTime_%': [], 'SourcebackPressuredTime_%': [], 'MappernumRecordsInPerSecond_avg'

In [22]:
itr=1
dvfs="1"
nrepeat = 2

for i in range(nrepeat):
    for rate in [100000, 200000, 300000]:
        for policy in ["powersave", "performance"]:
            parseFile(rate, itr, dvfs, policy, i)
print(df_dict)

/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_powersavedvfs1_repeat0/summary.csv
719
/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_performancedvfs1_repeat0/summary.csv
706
/home/han/github/logs/query1_cores16_frate200000_600000_fbuff-1_itr1_powersavedvfs1_repeat0/summary.csv
717
/home/han/github/logs/query1_cores16_frate200000_600000_fbuff-1_itr1_performancedvfs1_repeat0/summary.csv
706
/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_powersavedvfs1_repeat0/summary.csv
719
/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_performancedvfs1_repeat0/summary.csv
700
/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_powersavedvfs1_repeat1/summary.csv
718
/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_performancedvfs1_repeat1/summary.csv
705
/home/han/github/logs/query1_cores16_frate200000_600000_fbuff-1_itr1_powersavedvfs1_repeat1/summary.csv
718
/home/han/github/log

In [50]:
d = pd.DataFrame(df_dict)
print(d.columns)
d0 = d[(d.i == 0)]
#print(d.groupby(i))
d0[['i', 'rate', 'policy', 
   'SourcenumRecordsOutPerSecond_avg', 'SourcenumRecordsOutPerSecond_std', 'MapperbusyTimeMsPerSecond_avg',
   'watts_avg', 'rxBytes', 'txBytes']]

Index(['i', 'itr', 'dvfs', 'rate', 'policy', 'watts_avg', 'watts_std',
       'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'SinknumRecordsInPerSecond_avg',
       'SinknumRecordsInPerSecond_std', 'SinknumRecordsOutPerSecond_avg',
       'SinknumRecordsOutPerSecond_std', 'SinkbusyTimeMsPerSecond_avg',
       'SinkbusyTimeMsPerSecond_std', 'SinkbackPressuredTimeMsPerSecond_avg',
       'SinkbackPressuredTimeMsPerSecond_std', 'SinkbusyTime_%',
       'SinkbackPressuredTime_%', 'SourcenumRecordsInPerSecond_avg',
       'SourcenumRecordsInPerSecond_std', 'SourcenumRecordsOutPerSecond_avg',
       'SourcenumRecordsOutPerSecond_std', 'SourcebusyTimeMsPerSecond_avg',
       'SourcebusyTimeMsPerSecond_std',
       'SourcebackPressuredTimeMsPerSecond_avg',
       'SourcebackPressuredTimeMsPerSecond_std', 'SourcebusyTime_%',
       'SourcebackPressuredTime_%', 'MappernumRecordsInPerSecond_avg',
       'MappernumRecordsInPerSecond_std', 'Ma

,i,rate,policy,SourcenumRecordsOutPerSecond_avg,SourcenumRecordsOutPerSecond_std,MapperbusyTimeMsPerSecond_avg,watts_avg,rxBytes,txBytes
0,0,100000,powersave,100000.000000,0.000000,459.331250,45.19,29612586601,11990886019
1,0,100000,performance,99982.635159,104.574127,192.104167,47.92,29194546985,11835025938
2,0,200000,powersave,199980.334325,68.602511,862.362500,62.59,59776528897,23978155826
3,0,200000,performance,200000.000000,0.000000,404.791667,70.40,58773537527,23615684821
4,0,300000,powersave,224137.086508,2389.277311,960.333333,65.53,67093903287,26923316664
5,0,300000,performance,299976.178730,188.064980,574.983333,95.44,87490379821,35097132974


In [47]:
# d1 = d[(d.i == 1)]
#print(d.groupby(i))
d1[['i', 'rate', 'policy', 
   'SourcenumRecordsOutPerSecond_avg', 'MapperbusyTimeMsPerSecond_avg', 
   'watts_avg', 'rxBytes', 'txBytes']]

,i,rate,policy,SourcenumRecordsOutPerSecond_avg,MapperbusyTimeMsPerSecond_avg,watts_avg,rxBytes,txBytes
6,1,100000,powersave,99994.334881,455.545833,45.22,29555190640,11967485023
7,1,100000,performance,99998.804762,190.712500,47.34,29058370015,11776306304
8,1,200000,powersave,199973.677183,855.631250,62.37,59741522727,23966260985
9,1,200000,performance,200000.000000,431.841667,70.89,58467372578,23461272011
10,1,300000,powersave,201849.239365,904.237500,64.07,60396134509,24241588212
11,1,300000,performance,299983.821032,586.164583,94.20,88490680967,35481701071


In [51]:
print(67093903287/87490379821)

0.7668717797804747


In [52]:
print(26923316664/35097132974)

0.7671087175110521


In [53]:
print(65.53/95.44)

0.6866093880972339


In [54]:
print(224137.08650/299976.178730)

0.747182951156063
